# Training Machine Learning Models

In [6]:
import os

In [7]:
from pycaret.classification import *
from sklearn.model_selection import train_test_split
import pandas as pd

In [8]:
df = pd.read_parquet('datasets/teams/battlelog_train.parquet')
df.info()

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/teams/battlelog_train.parquet'

In [ ]:
# cambiar working directory
print(os.getcwd())
os.chdir('machine_learning')
print(os.getcwd())

c:\Users\alniquia\OneDrive - Telefonica\Documents\Projects\BrawlStars_Model
c:\Users\alniquia\OneDrive - Telefonica\Documents\Projects\BrawlStars_Model\machine_learning


In [ ]:
# generar set de testing y training
seed=14697

train, test = train_test_split(df, test_size = 0.7, random_state=seed)

train.shape

In [ ]:
# setup model
session_1 = setup(
	data = train,
	target = 'winner',
	fix_imbalance = True,
    feature_selection= True,
	remove_outliers=True,
	log_experiment = True,
    use_gpu=False)

NameError: name 'train' is not defined

In [ ]:
#comparación de modelos
model = compare_models(include=['rf','gbc','lightgbm','ada','catboost','xgboost'])

In [ ]:
#try model blender

# example:
# train a few models
# lr = create_model('lr')
# dt = create_model('dt')
# knn = create_model('knn')

# blend models
# blender = blend_models([lr, dt, knn])
# blender_tuned = tune_model(blender, optimize = 'F1')

In [ ]:
# tunear el mejor modelo
model_tuned = tune_model(model, optimize = 'F1')

In [ ]:
# interpret model
interpret_model(model_tuned)

In [ ]:
plot_model(model_tuned, plot='confusion_matrix', save=True)
plot_model(model_tuned, plot='feature', save=True)
plot_model(model_tuned, plot='calibration', save=True)

In [ ]:
# finalizar modelo
model_finalized = finalize_model(model_tuned)
predictions = predict_model(model_finalized, data = test)

In [ ]:
save_model(model_finalized, 'bs_predictor')